In [71]:
%matplotlib inline

import numpy as np
import scipy
import os
from astropy.io import fits
from ctisim.utils import ITL_AMP_GEOM, E2V_AMP_GEOM, OverscanParameterResults
from ctisim.fitting import OverscanFitting, BiasDriftSimpleModel

## Overscan Fitting Task

In [72]:
## Config variables
start = 3
stop = 15
ccd_type = 'itl'
max_signal = 140000.
min_signal = 20000.

## User provided variables
read_noise = 7.0
sensor_id = 'R20_S02'
overscan_results_file = '/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/BOT/6790D_linearity/R20/S02/R20_S02_overscan_results.fits'
output_dir = './'

In [73]:
cti_results = {i : 0.0 for i in range(1, 17)}
drift_scales = {i : 0.0 for i in range(1, 17)}
decay_times = {i : 0.0 for i in range(1, 17)}

if ccd_type == 'itl':
    ncols = ITL_AMP_GEOM.nx + ITL_AMP_GEOM.prescan_width
if ccd_type == 'e2v':
    ncols = E2V_AMP_GEOM.nx + E2V_AMP_GEOM.prescan_width

hdulist = fits.open(overscan_results_file)
    
for amp in range(1, 17):
    
    signals_all = hdulist[amp].data['FLATFIELD_SIGNAL']
    data_all = hdulist[amp].data['COLUMN_MEAN'][:, ncols+start-1:ncols+stop]
    indices = (signals_all < max_signal)*(signals_all>min_signal)
    signals = signals_all[indices]
    data = data_all[indices]
    
    params0 = [-7., 2.0, 2.5]
    constraints = [(-7., -7.), (0., 10.), (0.01, 4.0)]
    
    fitting_task = OverscanFitting(params0, constraints, BiasDriftSimpleModel,
                                   start=start, stop=stop)
    fit_results = scipy.optimize.minimize(fitting_task.negative_loglikelihood,
                                          params0,
                                          args=(signals, data, read_noise/np.sqrt(2000.), ncols),
                                          bounds=constraints, method='SLSQP')
    
    if fit_results.success:
        ctiexp, drift_scale, decay_time = fit_results.x
        cti_results[amp] = 10**ctiexp
        drift_scales[amp] = drift_scale/10000.
        decay_times[amp] = decay_time
        print(ctiexp, drift_scale, decay_time)
    else:
        num_failures += 1
        print(sensor_id, amp)
        print(fit_results)
        
outfile = os.path.join(output_dir, '{0}_overscan_fit_results.fits'.format(sensor_id))
parameter_results = OverscanParameterResults(sensor_id, cti_results, drift_scales, decay_times)
parameter_results.write_fits(outfile, overwrite=True)

-7.0 2.105900952867435 2.551489741421981
-7.0 2.2900137898102937 2.434242486580656
-7.0 2.271742272107894 2.3186322495077407
-7.0 2.0657611684712 2.4137333659471847
-7.0 1.996850955941276 2.554835932470865
-7.0 2.0677733393571054 2.5366589487312448
-7.0 2.1922221478097277 2.5417780761583804
-7.0 1.5482717840660407 2.257761376125488
-7.0 1.5958712936140138 2.418767435714549
-7.0 2.2338518525913345 2.5061217425337086
-7.0 2.1887768007593733 2.276517023303051
-7.0 2.027794960383091 2.4924283906068396
-7.0 1.901458792726938 2.4507042141320308
-7.0 1.8863409846296944 2.5647289938336773
-7.0 2.134677475193016 2.4603100711108308
-7.0 1.935922222112447 2.610053513135259


## CTI Fitting Task

In [74]:
## Config variables
start = 1
stop = 4
ccd_type = 'itl'
max_signal = 5000.

## User provided variables
read_noise = 7.0
sensor_id = 'R20_S02'
overscan_results_file = '/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/BOT/6790D_linearity/R20/S02/R20_S02_overscan_results.fits'
parameter_results_file = outfile # file from above
output_dir = './'

In [75]:
cti_results = {i : 0.0 for i in range(1, 17)}

hdulist = fits.open(overscan_results_file)
mcmc_amps = []

if ccd_type == 'itl':
    ncols = ITL_AMP_GEOM.nx + ITL_AMP_GEOM.prescan_width
if ccd_type == 'e2v':
    ncols = E2V_AMP_GEOM.nx + E2V_AMP_GEOM.prescan_width
    
parameter_results = OverscanParameterResults.from_fits(parameter_results_file)

for amp in range(1, 17):
    
    signals_all = hdulist[amp].data['FLATFIELD_SIGNAL']
    data_all = hdulist[amp].data['COLUMN_MEAN'][:, ncols+start-1:ncols+stop]
    indices = (signals_all < max_signal)
    signals = signals_all[indices]
    data = data_all[indices]
    
    output_amplifier = parameter_results.single_output_amplifier(amp, 1.)
    drift_scale = output_amplifier.scale*10000.
    decay_time = output_amplifier.decay_time
    
    params0 = [-6., drift_scale, decay_time]
    constraints = [(-6.8, -5.5), (drift_scale, drift_scale), (decay_time, decay_time)]
    
    fitting_task = OverscanFitting(params0, constraints, BiasDriftSimpleModel,
                                   start=start, stop=stop)
    fit_results = scipy.optimize.minimize(fitting_task.negative_loglikelihood,
                                          params0,
                                          args=(signals, data, read_noise/np.sqrt(2000.), ncols),
                                          bounds=constraints, method='SLSQP')
    
    if not fit_results.success:
        num_failures += 1
        print(sensor_id, amp)
        print(fit_results)
    else:
        ctiexp, drift_scale, decay_time = fit_results.x
        if  ctiexp > -5.6:
            mcmc_amps.append(amp)
            print(sensor_id, amp)
            print("Flagged for MCMC.")
        else:
            cti_results[amp] = 10**ctiexp
            print(ctiexp, drift_scale, decay_time)
            
parameter_results.cti_results = cti_results
parameter_results.write_fits(outfile, overwrite=True)

-6.277654788704249 2.105900930473581 2.55148983001709
-6.201521497656633 2.2900137992110103 2.4342424869537354
-6.0693723498265095 2.2717422689311206 2.3186323642730713
-6.108032745199523 2.0657612185459584 2.41373348236084
-6.148689373396013 1.9968509150203317 2.5548360347747803
-6.214019557618906 2.0677733118645847 2.536659002304077
R20_S02 7
Flagged for MCMC.
-6.365799820235675 1.5482718299608678 2.2577614784240723
R20_S02 9
Flagged for MCMC.
-5.953530383968964 2.2338518465403467 2.5061216354370117
-6.164439344934131 2.1887768525630236 2.276516914367676
-6.071011160392802 2.0277949806768447 2.4924283027648926
-6.172733358823932 1.9014587451238185 2.4507040977478027
-6.15372403076951 1.8863410514313728 2.5647289752960205
-6.285570785334125 2.1346774883568287 2.4603099822998047
-6.190463606418635 1.935922191478312 2.610053539276123


## Low Trap MCMC Fitting

In [76]:
from ctisim.fitting import JointSimulatedModel
from ctisim.core import LinearTrap
from ctisim.utils import save_mcmc_results
import emcee


## Config variables
start = 1
stop = 4
ccd_type = 'itl'
max_signal = 5000.

## User provided variables
read_noise = 7.0
sensor_id = 'R20_S02'
overscan_results_file = '/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/BOT/6790D_linearity/R20/S02/R20_S02_overscan_results.fits'
parameter_results_file = outfile # file from above
output_dir = './'
nwalkers = 8
nsteps = 100
burnin = 10

In [77]:
hdulist = fits.open(overscan_results_file)
parameter_results = OverscanParameterResults.from_fits(parameter_results_file)

for amp in mcmc_amps:
    
    signals_all = hdulist[amp].data['FLATFIELD_SIGNAL']
    data_all = hdulist[amp].data['COLUMN_MEAN'][:, ncols+start-1:ncols+stop]
    indices = (signals_all < max_signal)
    signals = signals_all[indices]
    data = data_all[indices]   
    
    output_amplifier = parameter_results.single_output_amplifier(amp, 1.)
    
    ## MCMC Fit
    params0 = [-6.0, 3.5, 0.4, 0.08]
    constraints = [(-7, -5.3), (0., 10.), (0.01, 1.0), (0.001, 1.0)]
    overscan_fitting = OverscanFitting(params0, constraints, JointSimulatedModel, start=start, stop=stop)

    scale = (0.4, 0.5, 0.05, 0.005)
    pos = overscan_fitting.initialize_walkers(scale, nwalkers)
    
    args = (signals, data, read_noise/np.sqrt(2000.), ITL_AMP_GEOM, LinearTrap, output_amplifier)
    sampler = emcee.EnsembleSampler(nwalkers, len(params0), overscan_fitting.logprobability, args=args)
    sampler.run_mcmc(pos, nsteps)
    
    save_mcmc_results(sensor_id, amp, sampler.chain, outfile, LinearTrap)
    
    ctiexp = np.median(sampler.chain[:, burnin:, 0])
    print(ctiexp)
    parameter_results.cti_results[amp] = 10**ctiexp
    
parameter_results.write_fits(outfile, overwrite=True)

-5.764386278103477
-5.928356689330895


In [78]:
parameter_results = OverscanParameterResults.from_fits(parameter_results_file)

print(parameter_results.cti_results)
print(parameter_results.drift_scales)
print(parameter_results.decay_times)

{1: 5.276491e-07, 2: 6.2875074e-07, 3: 8.52369e-07, 4: 7.797713e-07, 5: 7.100855e-07, 6: 6.109145e-07, 7: 1.7203378e-06, 8: 4.307251e-07, 9: 1.1793517e-06, 10: 1.1129346e-06, 11: 6.847951e-07, 12: 8.4915865e-07, 13: 6.7184124e-07, 14: 7.019012e-07, 15: 5.181186e-07, 16: 6.449654e-07}
{1: 0.0002105901, 2: 0.00022900138, 3: 0.00022717423, 4: 0.00020657612, 5: 0.00019968509, 6: 0.00020677733, 7: 0.00021922222, 8: 0.00015482718, 9: 0.00015958713, 10: 0.00022338518, 11: 0.00021887769, 12: 0.0002027795, 13: 0.00019014587, 14: 0.0001886341, 15: 0.00021346775, 16: 0.00019359222}
{1: 2.5514898, 2: 2.4342425, 3: 2.3186324, 4: 2.4137335, 5: 2.554836, 6: 2.536659, 7: 2.541778, 8: 2.2577615, 9: 2.4187675, 10: 2.5061216, 11: 2.276517, 12: 2.4924283, 13: 2.450704, 14: 2.564729, 15: 2.46031, 16: 2.6100535}
